## Turbofan Engine Degradation Simulation Dataset with Neural Networks

In [1]:
%matplotlib inline
from IPython.display import Image
from load_dataset import Load_Dataset
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import numpy as np
from math import sqrt
from sklearn.linear_model import Lasso, Ridge
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, TimeDistributed, RepeatVector,Flatten, Input, Dropout, LSTM, concatenate, Reshape, Conv1D, GlobalMaxPool1D
import keras.backend as K
from keras.utils import plot_model

Using TensorFlow backend.
/Users/johnmollas/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:493: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/johnmollas/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:494: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/johnmollas/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:495: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/johnmollas/anaconda3/lib/python3.6

In [2]:
#from wandb import magic #For visualization

First of all, we load and clean our data.

In [3]:
fm, feature_names = Load_Dataset.load_data_turbofan(False)

fm1_train = fm['FaultMode1']['df_train']
fm1_train_target = fm1_train['RUL'].values
fm1_test= fm['FaultMode1']['df_test']
fm1_test_target = fm1_test['RUL'].values

We are dropping some unecessary features.

In [4]:
LSTM_train = fm1_train.drop(columns=['t', 'os_1', 'os_2', 'os_3', 's_01', 's_05', 's_06', 's_10', 's_16', 's_18', 's_19', 's_22', 's_23', 's_24', 's_25', 's_26'])
LSTM_test = fm1_test.drop(columns=['t', 'os_1', 'os_2', 'os_3', 's_01', 's_05', 's_06', 's_10', 's_16', 's_18', 's_19', 's_22', 's_23', 's_24', 's_25', 's_26'])

We collect the different units, in order to the next steps to create time windows

In [5]:
train_units = set(LSTM_train['u'].values)
test_units = set(LSTM_test['u'].values)

We are scaling our data per feature

In [6]:
to_scale = ['s_02', 's_03', 's_04', 's_07', 's_08', 's_09', 's_11', 's_12',
            's_13', 's_14', 's_15', 's_17', 's_20', 's_21']
scalers = {}
for column in to_scale:
    scaler = MinMaxScaler(feature_range=(0.1,1.1))
    LSTM_train[column] = scaler.fit_transform(LSTM_train[column].values.reshape(-1,1))
    LSTM_test[column] = scaler.transform(LSTM_test[column].values.reshape(-1,1))
    scalers[column] = scaler

We create time windows with a specific size. In this example, we create time windows of 50 timesteps.

In [7]:
unit_scalers = {}
window = 50
temp_LSTM_x_train = []
LSTM_y_train = []
for unit in train_units:
    temp_unit = LSTM_train[LSTM_train['u']==unit].drop(columns=['u','RUL']).values
    temp_unit_RUL = LSTM_train[LSTM_train['u']==unit]['RUL'].values
    
    for i in range(len(temp_unit) - window + 1):#elekse edw an len temp_unit - window > 0
        temp_instance = []
        for j in range(window):
            temp_instance.append(temp_unit[i+j])
        temp_LSTM_x_train.append(np.array(temp_instance))
        LSTM_y_train.append(temp_unit_RUL[i+window-1])
LSTM_y_train = np.array(LSTM_y_train)
LSTM_x_train = np.array(temp_LSTM_x_train)

temp_LSTM_x_test = []
LSTM_y_test = []
for unit in test_units:
    temp_unit = LSTM_test[LSTM_test['u']==unit].drop(columns=['u','RUL']).values
    temp_unit_RUL = LSTM_test[LSTM_test['u']==unit]['RUL'].values
        
    for i in range(len(temp_unit) - window + 1):#elekse edw an len temp_unit - window > 0
        temp_instance = []
        for j in range(window):
            temp_instance.append(temp_unit[i+j])
        temp_LSTM_x_test.append(np.array(temp_instance))
        LSTM_y_test.append(temp_unit_RUL[i+window-1])
LSTM_y_test = np.array(LSTM_y_test)
LSTM_x_test = np.array(temp_LSTM_x_test)

We can check how many train, test instances we have. These are changing regarding the time window size.

In [8]:
LSTM_x_train.shape, LSTM_x_test.shape, LSTM_y_train.shape, LSTM_y_test.shape

((15731, 50, 14), (8255, 50, 14), (15731,), (8255,))

Moreover, we scale our target data, in order to train our models faster

In [9]:
temp_LSTM_y_train = [[i] for i in LSTM_y_train]
temp_LSTM_y_test = [[i] for i in LSTM_y_test]
target_scaler = MinMaxScaler()
target_scaler.fit(temp_LSTM_y_train)
temp_LSTM_y_train = target_scaler.transform(temp_LSTM_y_train)
temp_LSTM_y_test = target_scaler.transform(temp_LSTM_y_test)

We need a rmse loss function too!

In [10]:
def root_mean_squared_error(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true))) 

Now, we can build our predictor

In [11]:
feature_names = fm1_train.columns
encoder_input = Input(shape=(LSTM_x_train[0].shape))

encoder_x = LSTM(units=80, return_sequences=True, activation='tanh')(encoder_input)
encoder_x = Dropout(0.5)(encoder_x)
encoder_x = LSTM(units=40, return_sequences=False, activation='tanh')(encoder_x)

encoder_y = Conv1D(filters=40,kernel_size=3,activation='tanh')(encoder_input)
encoder_y = GlobalMaxPool1D()(encoder_y)

encoded = concatenate([encoder_x,encoder_y])
encoded = Dropout(0.5)(encoded)
encoded = Dense(80, activation='tanh')(encoded)#Relu and selu
encoded = Dropout(0.5)(encoded)
encoded = Dense(40, activation='tanh')(encoded)#Relu and selu
predictions = Dense(1, activation='sigmoid')(encoded)#Relu and selu
predictor = Model(encoder_input,predictions)

predictor.compile(optimizer="adam",loss=[root_mean_squared_error],metrics=['mae','mse'])
#print(predictor.summary())

checkpoint_name = 'TEDS_Predictor_Weights.hdf5' 
checkpoint = ModelCheckpoint(checkpoint_name, monitor='val_loss', verbose = 2, save_best_only = True, mode ='auto')

We can plot it as well!

In [12]:
from keras.utils import plot_model
plot_model(predictor, show_shapes=True, to_file='TEDS.png')

Then, we train the predictor.

In [13]:
#predictor.fit(LSTM_x_train, temp_LSTM_y_train, epochs=250, batch_size=512, shuffle=True, validation_split=0.33, verbose=2, callbacks=[checkpoint])

We load our weights, and we measure the performance.

In [14]:
wights_file = 'TEDS_Predictor_Weights.hdf5' # choose the best checkpoint few features
predictor.load_weights(wights_file) # load it
predictor.compile(optimizer="adam",loss=[root_mean_squared_error],metrics=['mae','mse'])

In [15]:
temp_pred = target_scaler.inverse_transform(predictor.predict(LSTM_x_train))
predictions = [i[0] for i in temp_pred]
print('Train:',mean_absolute_error(LSTM_y_train,predictions),mean_squared_error(LSTM_y_train,predictions),sqrt(mean_squared_error(LSTM_y_train,predictions)))
print(r2_score(LSTM_y_train,predictions))
temp_pred = target_scaler.inverse_transform(predictor.predict(LSTM_x_test))
predictions = [i[0] for i in temp_pred]
print('Test:',mean_absolute_error(LSTM_y_test,predictions),mean_squared_error(LSTM_y_test,predictions),sqrt(mean_squared_error(LSTM_y_test,predictions)))
print(r2_score(LSTM_y_test,predictions))

Train: 13.686472375760749 456.94272182388676 21.37621860441848
0.861136439080712
Test: 22.81737622261625 1048.122686165827 32.37472295118256
0.6154013922038681
